# TabNet + mBERT



## 前処理

In [3]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:7891"
os.environ["https_proxy"] = "http://127.0.0.1:7891"

In [4]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm.auto import tqdm
import re

In [5]:
# データ読み込み
train = pd.read_csv("./train/train.csv")
test = pd.read_csv("./test/test.csv")
anime = pd.read_csv("./train/anime.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [6]:
# 前処理コード
def scaling(data):
    sc = StandardScaler()
    data_sc = np.log1p(data)
    data_sc = sc.fit_transform(data_sc)
    return data_sc

class AnimePreprocessTabNet(object):
    def __init__(self, df, mode="train"):
        self.df = df
        self.mode = mode
        # 前処理関数一覧
        self.preprocessing_methods = [
            "anime_id",
            "genres",
            "japanese_name",
            "type",
            "episodes",
            "aired",
            "producers",
            "licensors",
            "studios",
            "source",
            "duration",
            "rating",
            "members",
            "watching",
            "completed",
            "on_hold",
            "dropped",
            "plan_to_watch",
            "concat_string_feature"
        ]

    def preprocess_all(self):
        # すべての処理を実行する
        for method_name in self.preprocessing_methods:
            getattr(self, method_name)()
        return self.df

    def anime_id(self):...
    
    def genres(self):
        """Create 26-dim embedding"""
        chars = ['Comedy', 'Sci-Fi', 'Seinen', 'Slice of Life', 'Space',
        'Adventure', 'Mystery', 'Historical', 'Supernatural', 'Fantasy',
        'Ecchi', 'School', 'Harem', 'Romance', 'Shounen', 'Action',
        'Magic', 'Sports', 'Super Power', 'Drama', 'Thriller', 'Music',
        'Shoujo', 'Demons', 'Mecha', 'Game', 'Josei', 'Cars',
        'Psychological', 'Parody', 'Samurai', 'Military', 'Shoujo Ai',
        'Kids', 'Martial Arts', 'Horror', 'Dementia', 'Vampire',
        'Shounen Ai', 'Hentai', 'Yaoi', 'Police']
        genres = self.df[['anime_id','genres']]
        genres.loc[:,chars] = 0
        genres['genres'] = genres['genres'].str.split(',')
        # genres[chars] = 0
        for i, row in genres.iterrows():
            for index in (s.strip() for s in row['genres']):
                    genres.loc[i,index] = 1
        genres = genres.drop('genres',axis=1)
        self.df = pd.merge(self.df, genres, on="anime_id", how="left")
    
    def japanese_name(self):...
    
    def type(self):
        # 単純にラベルエンコーディング
        encoder = LabelEncoder()
        self.df["type"] = encoder.fit_transform(self.df["type"])
    
    def episodes(self):
        unknown_mask = self.df["episodes"] == "Unknown"
        self.df.loc[~unknown_mask, "episodes"] = scaling(
            self.df.loc[~unknown_mask, "episodes"].astype(int).to_numpy().reshape(-1, 1)
        )
        self.df.loc[unknown_mask, "episodes"] = -1
        self.df["episodes"] = self.df["episodes"].astype(float)
    def to_year(self,s):
        match = re.search(r'\d{4}', s)
        if match:
            return int(match.group())
        else:
            return 1000
    def aired(self):
        # 扱いにくそうなので、一度落とす
        self.df['year'] = self.df['aired'].apply(self.to_year)
        self.df = self.df.drop(columns=["aired"])

    def producers(self):
        """制作会社の総数を算出する。また、後で言語変数としても使う
        """
        self.df["num_producers"] = self.df["producers"].str.split(",").str.len()
        # 対数変換→ 標準化
        self.df["num_producers"] = scaling(self.df[["num_producers"]].to_numpy())
    
    def licensors(self):...
    
    def studios(self):
        # 単純にラベルエンコーディング
        encoder = LabelEncoder()
        self.df["studios"] = encoder.fit_transform(self.df["studios"])
    
    def source(self):
        # 単純にラベルエンコーディング
        encoder = LabelEncoder()
        self.df["source"] = encoder.fit_transform(self.df["source"])
    
    def duration(self):
        unknown_mask = self.df["duration"] == "Unknown"
        self.df.loc[self.df["duration"].str.contains("hr"), "duration"] = \
            self.df.loc[self.df["duration"].str.contains("hr"), "duration"]\
                    .str.extract("(\d+)")[0].astype(float) * 60
        self.df["duration"] = self.df["duration"].str.extract("(\d+)")[0]
        self.df["duration"] = self.df["duration"].astype(float)
        self.df.loc[~unknown_mask, "duration"] = scaling(
            self.df.loc[~unknown_mask, "duration"].to_numpy().reshape(-1, 1)
        )
        self.df["duration"] = self.df["duration"].fillna(-1).astype(int)

    def rating(self):...
    
    def members(self):
        # 対数変換→ 標準化
        self.df["members"] = scaling(self.df[["members"]].to_numpy())
    
    def watching(self):
        # 対数変換→ 標準化
        self.df["watching"] = scaling(self.df[["watching"]].to_numpy())
    
    def completed(self):
        # 対数変換→ 標準化
        self.df["completed"] = scaling(self.df[["completed"]].to_numpy())
    
    def on_hold(self):
        # 対数変換→ 標準化
        self.df["on_hold"] = scaling(self.df[["on_hold"]].to_numpy())
    
    def dropped(self):
        # 対数変換→ 標準化
        self.df["dropped"] = scaling(self.df[["dropped"]].to_numpy())
    
    def plan_to_watch(self):
        # 対数変換→ 標準化
        self.df["plan_to_watch"] = scaling(self.df[["plan_to_watch"]].to_numpy())
    
    def concat_string_feature(self):
        # 文字列として扱う列を結合し、元の列を落とす
        concat_feature = [
            "japanese_name",
            "genres",
            "producers",
            "licensors",
            "studios",
            "rating"
        ]
        # スペース区切りで結合する
        self.df[concat_feature] = self.df[concat_feature].astype(str)
        self.df['combined_features'] = self.df[concat_feature].agg(' '.join, axis=1)
        # 元の列を落とす
        self.df = self.df.drop(columns=concat_feature)

airedについてもくっつけて一緒にembeddingすることができますが、このときは「なんとなく扱いにくそうだなあ(バイアスがかかりそうだなあ)」と思って一旦ドロップしちゃいました。  
使っても全く問題ないと思います。  
カテゴリカル変数はラベルエンコーディング、数値特徴量は基本的に対数変換→ 標準化を行い、欠損や欠損に該当しそうなUnknownという値は-1を入れています。(書いていて気づいたんですが、これだと異常値として認識させられていないですね。皆さんは気をつけましょう。。。)  
経験的に、以下の特徴量の作り方だとTabNetの学習がうまく行きやすい気がします。  
- カテゴリカル変数は基本ラベルエンコーディングのあとembedding
- 数値特徴量は対数変換→ 標準化
- 特徴量間の交互作用はTabNetが見つけてくれるので、自分で作成した交互作用を表すお気持ちの特徴量は全部落とす

LightGBMの後にTabNetを試されるケースが多いのかなと思うので、特に3つ目を意識すると性能を引き出しやすいと感じていますが、残念ながらn=1なので本当のところはよくわかりません。

In [7]:
anime_preprocess = AnimePreprocessTabNet(df=anime, mode="train")
x = anime_preprocess.preprocess_all()

C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_21652\1650585605.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres.loc[:,chars] = 0
C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_21652\1650585605.py:54: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  genres.loc[:,chars] = 0
C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_21652\1650585605.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

### mBERTによるembedding

今回は日本語と英語が混ざっている特徴量を扱うので、日本語特化 or 英語特化ではなく、多言語を学習したモデルが望ましいと思いました。  
そこでmultilingual BERTを使ってみます。  
なお、本notebookで扱うモデルであるTabNetとは、ライブラリの依存関係が面倒なため、mBERT、その他言語モデルによる特徴量埋め込みとTabNetは仮想環境を分けることをおすすめします。  
あと、このコードはChatGPTくんが8割くらい書いてくれました。  
書くのめんどいな～って思って投げると爆速で書いてくれるので本当にありがたい・・・

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from tqdm.notebook import tqdm

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            max_length=512,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze()
        }

class TextEmbedder:
    def __init__(self, device=None):
        self.model = BertModel.from_pretrained('model/mBert')
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        self.model.to(self.device)

    def get_embeddings(self, texts, batch_size=16):
        dataset = TextDataset(texts)
        dataloader = DataLoader(dataset, batch_size=batch_size)
        embeddings = []
        with torch.no_grad():
            for batch in tqdm(dataloader):
                inputs = {name: tensor.to(self.device) for name, tensor in batch.items()}
                outputs = self.model(**inputs)
                embeddings.append(outputs.last_hidden_state.mean(dim=1).cpu().numpy())
        embeddings = np.concatenate(embeddings)
        return embeddings


embeddingを作っていきます。  
RTX3070のミドルエンドGPUでも、このデータ規模ならすぐ終わります。

In [ ]:
# encodingする
embedder = TextEmbedder()
embeddings = embedder.get_embeddings(anime_preprocess.df['combined_features'].values.tolist())

TabNetと環境を分ける都合上、保存しておいてください。  
本来はここでnotebookも一度分かれます。

In [9]:
# 特徴量は保存しておく
with open("./train/mBERT_embedding_01EDA.npy", "wb") as f:
    np.save(f, embeddings)


あとはこれをくっつけたらOKです。  

In [7]:
embeddings = np.load("./train/mBERT_embedding_01EDA.npy")

In [11]:
df_anime_preprocessd = anime_preprocess.df.copy()
df_anime_preprocessd = df_anime_preprocessd.drop(columns=["combined_features"])
embeddings_columns = [f"mBERT_{i}" for i in range(embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=embeddings, columns=embeddings_columns)
df_anime_preprocessd = df_anime_preprocessd.join(embeddings_df)

In [12]:
df_anime_preprocessd.head(3)

,anime_id,type,episodes,source,duration,members,watching,completed,on_hold,dropped,...,mBERT_758,mBERT_759,mBERT_760,mBERT_761,mBERT_762,mBERT_763,mBERT_764,mBERT_765,mBERT_766,mBERT_767
0,000ba7f7e34e107e7544,5,2.242456,6,0,-0.190714,0.582513,-0.510020,1.007777,0.407904,...,-0.205883,-0.231246,-0.598498,-0.532398,0.332631,-0.144089,0.203178,0.709978,0.144264,0.346295
1,00427279d72064e7fb69,5,0.942888,6,0,1.601196,1.467724,0.610288,1.937989,1.090300,...,-0.017266,-0.192892,-0.581323,-0.603976,0.004401,-0.217524,0.190385,0.566311,0.227716,0.365409
2,00444b67aaabdf740a68,5,0.051643,6,0,0.326796,0.381992,0.167470,0.415445,-0.224419,...,-0.010629,-0.248850,-0.683986,-0.542475,0.118411,0.109113,0.173260,0.666322,0.347690,0.192940


一応コサイン類似度を確認して、「近い作品」が「似ている」のか確かめてみたいと思います。

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_matrix = cosine_similarity(embeddings)

# サンプルの作品index
sample_index = 18
cosine_sim_scores = cosine_sim_matrix[sample_index]
sorted_indices = np.argsort(cosine_sim_scores)[::-1]

print("Sample text:")
print(anime_preprocess.df["combined_features"].to_numpy()[sample_index])
print("\nMost similar texts:")
for i in sorted_indices[1:11]:
    print(anime_preprocess.df["combined_features"].to_numpy()[i])

Sample text:
やはり俺の青春ラブコメはまちがっている。続 Slice of Life, Comedy, Drama, Romance, School TBS, Marvelous AQL, NBCUniversal Entertainment Japan Sentai Filmworks 266 PG-13 - Teens 13 or older

Most similar texts:
やはり俺の青春ラブコメはまちがっている。完 Slice of Life, Comedy, Drama, Romance, School Marvelous, TBS, Movic, Delfi Sound, NBCUniversal Entertainment Japan Sentai Filmworks 266 PG-13 - Teens 13 or older
アウトブレイク・カンパニー Harem, Comedy, Parody, Fantasy Pony Canyon, TBS, Kodansha, Movic, DAX Production Sentai Filmworks 266 PG-13 - Teens 13 or older
パパのいうことを聞きなさい！ Comedy, Romance, Slice of Life Starchild Records, KlockWorx, PPP, Studio Mausu, Shueisha Sentai Filmworks 266 PG-13 - Teens 13 or older
マンガ家さんとアシスタントさんと THE ANIMATION Harem, Slice of Life, Comedy, Ecchi, Seinen Lantis, Magic Capsule, Showgate Sentai Filmworks 265 PG-13 - Teens 13 or older
やはり俺の青春ラブコメはまちがっている。 Slice of Life, Comedy, Drama, Romance, School Geneon Universal Entertainment, TBS, Delfi Sound, Marvelous AQL, Atelier Musa Sentai Filmworks 41 PG

「俺ガイル」がそのシリーズと近いところにいます。  
でも他はどうなんだろうか。  
微妙な気もしますね。モデルの限界かもしれません。  
実はこの後、E5のモデルでも埋め込みを試しているので、そちらもコードと結果を載せておきます。  

In [14]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

class TextDataset(Dataset):
    def __init__(self, texts, model_name='intfloat/multilingual-e5-small'):
        self.texts = texts
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            max_length=512,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze()
        }

class TextEmbedder:
    def __init__(self, device=None, model_name='model/multilingual-e5-small'):
        self.model_name = model_name
        self.model = AutoModel.from_pretrained(self.model_name)
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        self.model.to(self.device)

    def average_pool(self, last_hidden_states, attention_mask):
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def get_embeddings(self, texts, batch_size=16):
        dataset = TextDataset(texts, self.model_name)
        dataloader = DataLoader(dataset, batch_size=batch_size)
        embeddings = []
        with torch.no_grad():
            for batch in tqdm(dataloader):
                inputs = {name: tensor.to(self.device) for name, tensor in batch.items()}
                outputs = self.model(**inputs)
                embeddings.append(self.average_pool(outputs.last_hidden_state, inputs['attention_mask']).cpu().numpy())
        embeddings = np.concatenate(embeddings)
        return embeddings

In [17]:
# smallモデル
embedder = TextEmbedder()
embeddings_small = embedder.get_embeddings(anime_preprocess.df['combined_features'].values.tolist())

OSError: model/multilingual-e5-small is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [19]:
embeddings_small = np.load("train/mBERT_embedding_small.npy")

In [20]:
cosine_sim_matrix = cosine_similarity(embeddings_small)

# サンプルの作品index
sample_index = 18
cosine_sim_scores = cosine_sim_matrix[sample_index]
sorted_indices = np.argsort(cosine_sim_scores)[::-1]

print("Sample text:")
print(anime_preprocess.df["combined_features"].to_numpy()[sample_index])
print("\nMost similar texts:")
for i in sorted_indices[1:11]:
    print(anime_preprocess.df["combined_features"].to_numpy()[i])

Sample text:
やはり俺の青春ラブコメはまちがっている。続 Slice of Life, Comedy, Drama, Romance, School TBS, Marvelous AQL, NBCUniversal Entertainment Japan Sentai Filmworks 266 PG-13 - Teens 13 or older

Most similar texts:
やはり俺の青春ラブコメはまちがっている。完 Slice of Life, Comedy, Drama, Romance, School Marvelous, TBS, Movic, Delfi Sound, NBCUniversal Entertainment Japan Sentai Filmworks 266 PG-13 - Teens 13 or older
やはり俺の青春ラブコメはまちがっている。 Slice of Life, Comedy, Drama, Romance, School Geneon Universal Entertainment, TBS, Delfi Sound, Marvelous AQL, Atelier Musa Sentai Filmworks 41 PG-13 - Teens 13 or older
やはり俺の青春ラブコメはまちがっている。OVA「こちらとしても彼ら彼女らの行く末に幸多からんことを願わざるを得ない。」 Comedy, Romance, School Unknown Unknown 41 PG-13 - Teens 13 or older
やはり俺の青春ラブコメはまちがっている. 続 きっと, 女の子はお砂糖とスパイスと素敵な何かでできている。 Comedy, Romance, School 5pb. feel. 266 PG-13 - Teens 13 or older
僕らはみんな河合荘 Slice of Life, Comedy, Romance, School, Seinen TBS Sentai Filmworks 41 PG-13 - Teens 13 or older
坂本ですが？ Slice of Life, Comedy, School, Seinen TBS, DAX Production, Ki

In [15]:
# largeモデル
embedder = TextEmbedder(model_name='intfloat/multilingual-e5-large')
embeddings_large = embedder.get_embeddings(anime_preprocess.df['combined_features'].values.tolist())

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like intfloat/multilingual-e5-large is not the path to a directory containing a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [21]:
embeddings_large = np.load("train/mBERT_embedding_large.npy")


In [22]:
cosine_sim_matrix = cosine_similarity(embeddings_large)

# サンプルの作品index
sample_index = 18
cosine_sim_scores = cosine_sim_matrix[sample_index]
sorted_indices = np.argsort(cosine_sim_scores)[::-1]

print("Sample text:")
print(anime_preprocess.df["combined_features"].to_numpy()[sample_index])
print("\nMost similar texts:")
for i in sorted_indices[1:11]:
    print(anime_preprocess.df["combined_features"].to_numpy()[i])

Sample text:
やはり俺の青春ラブコメはまちがっている。続 Slice of Life, Comedy, Drama, Romance, School TBS, Marvelous AQL, NBCUniversal Entertainment Japan Sentai Filmworks 266 PG-13 - Teens 13 or older

Most similar texts:
やはり俺の青春ラブコメはまちがっている。完 Slice of Life, Comedy, Drama, Romance, School Marvelous, TBS, Movic, Delfi Sound, NBCUniversal Entertainment Japan Sentai Filmworks 266 PG-13 - Teens 13 or older
やはり俺の青春ラブコメはまちがっている。 Slice of Life, Comedy, Drama, Romance, School Geneon Universal Entertainment, TBS, Delfi Sound, Marvelous AQL, Atelier Musa Sentai Filmworks 41 PG-13 - Teens 13 or older
やはり俺の青春ラブコメはまちがっている. 続 きっと, 女の子はお砂糖とスパイスと素敵な何かでできている。 Comedy, Romance, School 5pb. feel. 266 PG-13 - Teens 13 or older
アマガミSS+ plus Slice of Life, Comedy, Romance, School TBS Sentai Filmworks 9 PG-13 - Teens 13 or older
きんいろモザイク Slice of Life, Comedy, School, Seinen Media Factory, Showgate Sentai Filmworks 210 PG-13 - Teens 13 or older
カラフル Drama, Slice of Life, Supernatural Aniplex, Sony Music Entertainment, Imagine Se

全然様相が違うことがわかります。  
E5モデルのほうが、シリーズものが近しいところにいるので、より「らしい」特徴量なのかもしれません。  
「俺ガイル」でしかそのあたりは確認していないので、「俺ガイルベンチマーク」がどれだけ信憑性のあるものなのかは未知数です。  
余談ですが、僕はいろはすが好きです。

### train.csvの処理

In [23]:
class TrainPreprocess(object):
    def __init__(self, train, test, mode="train"):
        self.train = train
        self.test = test
        self.mode = mode
        self.df_all = pd.concat([train, test]).reset_index(drop=True)
        self.preprocessing_methods = [
            "user_id"
        ]

    def preprocess_all(self):
        # すべての処理を実行する
        for method_name in self.preprocessing_methods:
            getattr(self, method_name)()

    def user_id(self):
        # user_idのencoding
        self.encoder = LabelEncoder()
        self.encoder.fit(self.df_all["user_id"])
        self.train["user_id"] = self.encoder.transform(self.train["user_id"])
        self.test["user_id"] = self.encoder.transform(self.test["user_id"])

def merge_anime(df, anime):
    """trainまたはtestとanimeをmergeする
    """
    df_merge = pd.merge(df, anime, on=["anime_id"], how="left")
    return df_merge

In [24]:
def concat_train_test(train, test):
    train["train_test"] = "train"
    test["train_test"] = "test"
    df_all = pd.concat([train, test])
    return df_all

In [25]:
train_preprocess = TrainPreprocess(train=train, test=test)
train_preprocess.preprocess_all()
df_merge_train = merge_anime(train_preprocess.train, df_anime_preprocessd)
df_merge_test = merge_anime(train_preprocess.test, df_anime_preprocessd)
df_merge_all = concat_train_test(df_merge_train, df_merge_test)

In [32]:
df_merge_all.head()['year']

0    2018
1    2018
2    2006
3    2016
4    2003
Name: year, dtype: int64

ここはuser_idをラベルエンコーディングしてくっつけるだけです。  
このあとTabNetのPretrainを行なうためにtest データを繋げているんですが、こちらは本来未知なものではあるので、concatするかどうかは思想によるかもしれません。

## TabNetによる学習

ここからはTabNetによる学習を行っていきます。  

TabNetには、「カテゴリカル変数のembedding」という機能があるので、`user_id`, `type`, `source` の3つについて、埋め込みベクトルを作らせてみます。  
各次元数は結構feelingです。気になる方は色々試してみてください。  
なお、ここからの注意点として、Google Colab環境の場合、最初からinstallされているpytorchが邪魔をするので、以下でTabNetのライブラリを入れてください。  

```shell
pip uninstall torchdata  torchtext torchvision torchaudio
pip install pytorch-tabnet
```

In [27]:
import torch
import numpy as np
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error


class TabNetBaseline:
    def __init__(
        self,
        tabnet_params,
        embedding_cols,
        embedding_idx,
        cat_dims,
        embedding_dims,
        splitter=KFold(n_splits=5, shuffle=True, random_state=42),
        seed=777
        ):
        self.tabnet_params = tabnet_params
        self.embedding_cols = embedding_cols
        self.embedding_idx = embedding_idx
        self.cat_dims = cat_dims
        self.embedding_dims = embedding_dims
        self.splitter = splitter
        self.seed = seed
        self.models = []
        self.oof_preds = None
        self.pretrained_model = None

    def prepare_pretrain_data(self, df_pretrain):

        for embc, d in zip(self.embedding_cols, self.cat_dims):
            df_pretrain[embc] = df_pretrain[embc].replace({-1: df_pretrain[embc].max() + 1})
            assert (df_pretrain[embc].max()+1 == d)
            assert (len(df_pretrain[embc].unique()) == d)
            assert df_pretrain[embc].min()==0

        return df_pretrain

    def pretrain(self, df_pretrain):
        if not self.pretrained_model:
            df_pretrain = self.prepare_pretrain_data(df_pretrain)
            train_unsp, val_unsup = train_test_split(df_pretrain,  test_size=0.3, random_state=self.seed)

            unsupervised_model = TabNetPretrainer(
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=0.1),
                scheduler_params={"step_size":10, "gamma":0.9},
                cat_idxs=self.embedding_idx, 
                cat_dims=self.cat_dims,
                cat_emb_dim=self.embedding_dims,
                **self.tabnet_params
            )

            unsupervised_model.fit(
                X_train=train_unsp.values,
                eval_set=[val_unsup.values],
                pretraining_ratio=0.8,
                max_epochs=300
            )

            self.pretrained_model = unsupervised_model
        return self.pretrained_model

    def train(self, X, y, groups=None):
        self.models = []
        self.oof_preds = np.zeros(len(X))
        scores = []
        for train_index, valid_index in self.splitter.split(X, y, groups=groups):
            unsupervised_model = self.pretrained_model

            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

            reg = TabNetRegressor(
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=0.03),
                scheduler_params={"step_size":10, "gamma":0.9},
                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                cat_idxs=self.embedding_idx, 
                cat_dims=self.cat_dims,
                cat_emb_dim=self.embedding_dims,
                **self.tabnet_params
            )

            reg.fit(
                X_train=X_train.values, y_train=y_train.values.reshape(-1, 1),
                eval_set=[(X_train.values, y_train.values.reshape(-1, 1)), (X_valid.values, y_valid.values.reshape(-1, 1))],
                eval_name=['train', 'valid'],
                eval_metric=['rmse'],
                batch_size=2048, virtual_batch_size=2048,
                drop_last=True,
                from_unsupervised=unsupervised_model,
                max_epochs=1000,
                patience=12,
                num_workers=4,
            )
            self.models.append(reg)

            y_pred = reg.predict(X_valid.values)[:, 0]
            self.oof_preds[valid_index] = y_pred
            score = mean_squared_error(y_valid, y_pred, squared=False)  # RMSE score
            scores.append(score)
        self.cv_score = np.mean(scores)

    def inference(self, X):
        y_preds = []
        for model in self.models:
            y_pred = model.predict(X)[:, 0]
            y_preds.append(y_pred)
        y_preds = np.mean(y_preds, axis=0)
        return y_preds

    def plot_feature_importance(self):
        df_features_list = []
        for model in self.models:
            df = pd.DataFrame(data ={
                "feature_importance" : model.feature_importances_,
                "feature_names" : self.features
            })
            df_features_list.append(df)

        df_features = pd.concat(df_features_list).sort_values(by='feature_importance', ascending=False)

        f, ax = plt.subplots(figsize=(5, 10))
        sns.barplot(
            data = df_features,
            x = 'feature_importance',
            y = 'feature_names',
            capsize=0.1, errwidth=1.2,
            ax = ax
        )
        return f, ax

この辺のハイパーパラメータは調整すると色々結果が変わるので面白いです。

In [28]:
# Pretrainerのハイパーパラメータ
tabnet_params = {
    'mask_type' : 'entmax',
    'n_d' : 64,
    'n_a': 64,
    'n_steps': 3,
    'gamma': 0.9,
    'verbose': 10
}

Pretrainをしていきます。  
事前学習はしたほうがよいとTabNetでは言われているのでするお気持ちです。  
たぶん精度が変わると思います、たぶん。

In [29]:
drop_cols = ["anime_id", "score", "train_test"]
df_pretrain = df_merge_all.drop(columns=drop_cols) # 目的変数は使わないので落とす

# embeddingするカラム
embeding_cols = ['user_id', 'type', 'source']
# 各々のユニーク数
col_uniques = [ len(df_pretrain[c].unique()) for c in  embeding_cols ]
# カラムの番号
embeding_idx = [ i for i, c in  enumerate(df_pretrain.columns) if c in embeding_cols]
cat_dims = [ v for k, v in zip(embeding_cols, col_uniques) if k in embeding_cols]
# 埋め込み次元
embeding_dims = [ i // 2 if i<100 else 64 for i in cat_dims ]

In [35]:
cat_dims

[1998, 6, 13]

In [31]:
tabnet_mbert = TabNetBaseline(
    tabnet_params=tabnet_params,
    embedding_cols=embeding_cols,
    embedding_idx=embeding_idx,
    cat_dims=cat_dims,
    embedding_dims=embeding_dims
)
tabnet_mbert.pretrain(df_pretrain)

e:\Envs\kaggle1\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 106.19549| val_0_unsup_loss_numpy: 102.87663269042969|  0:00:32s
epoch 10 | loss: 5.39076 | val_0_unsup_loss_numpy: 52.667579650878906|  0:03:57s

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_0_unsup_loss_numpy = 36.580501556396484


e:\Envs\kaggle1\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNetPretrainer(n_d=64, n_a=64, n_steps=3, gamma=0.9, cat_idxs=[0, 1, 3], cat_dims=[1998, 6, 13], cat_emb_dim=[64, 3, 6], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=10, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.1}, scheduler_fn=None, scheduler_params={'step_size': 10, 'gamma': 0.9}, mask_type='entmax', input_dim=823, output_dim=None, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)

環境を切り替えて実行した結果をまとめるのが面倒だったので、学習時の出力をペタリ。

```shell
epoch 0  | loss: 4.61219 | val_0_unsup_loss_numpy: 0.8416699767112732|  0:00:21s
epoch 10 | loss: 0.65599 | val_0_unsup_loss_numpy: 0.5929800271987915|  0:03:44s

Early stopping occurred at epoch 16 with best_epoch = 6 and best_val_0_unsup_loss_numpy = 0.44297999143600464
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
```

Pretrain時にこのくらいまで収束してくれていると、性能が発揮しやすい印象です。  
保存すると、`.zip`形式になります。

In [ ]:
# 保存
tabnet_mbert.pretrained_model.save_model('../model/pretrain_mBERT')

最後に予測モデルを学習させていきます。

In [ ]:
train_x = df_merge_all.loc[df_merge_all["train_test"]=="train"].copy().drop(columns=drop_cols)
train_y = df_merge_all.loc[df_merge_all["train_test"]=="train"].copy()['score']

loaded_pretrain = TabNetPretrainer()
loaded_pretrain.load_model('../model/pretrain_mBERT.zip')
tabnet_mbert.pretrained_model = loaded_pretrain
tabnet_mbert.train(train_x, train_y)

出力はこんな感じです。  
```
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:231: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")
epoch 0  | loss: 8.73306 | train_rmse: 1.59975 | valid_rmse: 1.61224 |  0:00:11s
epoch 10 | loss: 1.21449 | train_rmse: 1.04232 | valid_rmse: 1.20917 |  0:02:04s
epoch 20 | loss: 0.76145 | train_rmse: 0.77771 | valid_rmse: 1.25488 |  0:03:57s

Early stopping occurred at epoch 24 with best_epoch = 12 and best_valid_rmse = 1.19892
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:231: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")
epoch 0  | loss: 7.54927 | train_rmse: 1.57612 | valid_rmse: 1.57344 |  0:00:11s
epoch 10 | loss: 1.53426 | train_rmse: 1.22548 | valid_rmse: 1.25262 |  0:02:03s
epoch 20 | loss: 1.03442 | train_rmse: 0.95992 | valid_rmse: 1.21042 |  0:03:55s

Early stopping occurred at epoch 26 with best_epoch = 14 and best_valid_rmse = 1.18938
```

Foldごとのスコア  
| Fold | validation rmse | 
| ---- | --------------- | 
| 0    | 1.19892         | 
| 1    | 1.18938         | 
| 2    | 1.19768         | 
| 3    | 1.19511         | 
| 4    | 1.19257         | 


In [14]:
test_x = df_merge_all.loc[df_merge_all["train_test"]=="test"].copy().drop(columns=drop_cols)

In [ ]:
pred = tabnet_mbert.inference(test_x.to_numpy())
sample_submission["score"] = pred
sample_submission.to_csv("../submission/06_tabnet_random.csv", index=False)

In [ ]:
# モデル保存
for i, regression_model in enumerate(tabnet_mbert.models):
    regression_model.save_model(f'../model/06_tabnet_random_{i}')

以上です。  
これ書いてたらもう73位まで落ちてました！！  
ここからスコア巻き返せるかな。俺たちの戦いはこれからだ！